### todo
- "newtrial_frame", we might not need it in the future
- save name

In [ ]:
import numpy as np
import pandas as pd
import scipy.io
from pathlib import Path
# from kilosort.io import load_ops

date = 20250710

# Define Path and Load Files
results_dir = Path(r'D:\Neural-Pipeline\data\20250602\kilosort4_phy')
# APtimestamps = np.load(r"D:\20250313\2025-03-13_15-09-41\Record Node 101\experiment1\recording1\continuous\Neuropix-PXI-100.ProbeA-AP\timestamps.npy")
# save_path = r'D:\20250313\zarya_20250313__bankAB_unit.mat'

# spike_clusters = np.load(results_dir / 'spike_clusters.npy')
spike_times = np.load(results_dir / 'spike_times.npy')
spike_pos = np.load(results_dir / 'spike_positions.npy')
channel_positions = np.load(results_dir / 'channel_positions.npy')

channel_map =  np.load(results_dir / 'channel_map.npy')
templates =  np.load(results_dir / 'templates.npy')

cluster_group = pd.read_csv(results_dir / 'cluster_info.tsv', sep='\t')
# good_count = (cluster_group["group"] == "good").sum()
# test = cluster_group["depth"][0]
# print(f"Number of 'good' clusters: {good_count}")
# print(test)

### Report number of unit 

In [13]:
import numpy as np
import pandas as pd
from pathlib import Path
phy = True

phy_dir = Path(r'D:\Neural-Pipeline\data\20250417\kilosort4_phy')
kilo_dir = Path(r'D:\Neural-Pipeline\data\20250417\kilosort4')
cluster_group = pd.read_csv(phy_dir / 'cluster_info.tsv', sep='\t')
kilo_cluster = pd.read_csv(kilo_dir / 'cluster_KSLabel.tsv', sep='\t')


# Count KSLabel (Kilosort output)
kslabel_counts = kilo_cluster['KSLabel'].value_counts(dropna=False)

# Count group (Phy manual curation)
if phy:
    group_counts = cluster_group['group'].value_counts(dropna=False)

# Report both
print("=== Kilosort Labels (KSLabel) ===")
print(kslabel_counts)
if phy: 
    print("\n=== Phy Curated Labels (group) ===")
    print(group_counts)


=== Kilosort Labels (KSLabel) ===
KSLabel
good    506
mua     411
Name: count, dtype: int64

=== Phy Curated Labels (group) ===
group
mua      429
noise    216
good     176
NaN        2
Name: count, dtype: int64


In [10]:
print(cluster_group)

     cluster_id  Amplitude  ContamPct KSLabel          amp   ch   depth  \
0             0       39.8        0.0    good  1760.941895    0     0.0   
1             1       21.7        0.0    good   976.867065    4    40.0   
2             2       11.0        6.2    good   517.755310    2    20.0   
3             3       18.1        0.0     mua   676.689575    2    20.0   
4             4       26.7        0.0    good  1274.200806    2    20.0   
..          ...        ...        ...     ...          ...  ...     ...   
884         884        9.3       16.0    good   267.885773  379  7620.0   
885         885       10.4       46.9     mua   349.544800  379  7620.0   
886         886       10.0       14.7    good   316.992188  383  7660.0   
887         887        9.5       33.3     mua   328.081604  381  7640.0   
888         888        9.3       29.5     mua   271.145691  381  7640.0   

           fr  group  n_spikes   sh  
0    0.024184    NaN       358  1.0  
1    0.012903    NaN   

In [ ]:
chan_best = (templates**2).sum(axis=1).argmax(axis=-1)
chan_best = channel_map[chan_best]
template_amplitudes = ((templates**2).sum(axis=(-2,-1))**0.5)
uniq_spike_clusters = np.unique(spike_clusters)
spike_counts = np.unique(spike_clusters, return_counts=True)[1]
channel_y_positions = channel_positions[:, 1]
newtrial_frame = np.where(np.diff(APtimestamps) < -1) [0]
print(newtrial_frame)

In [8]:
## 20250313 version
uniq_spike_clusters = np.unique(spike_clusters)
depth = np.zeros(len(uniq_spike_clusters))
group = np.zeros(len(uniq_spike_clusters))
cluster_id = np.zeros(len(uniq_spike_clusters))
ch = np.zeros(len(uniq_spike_clusters))
spiketimes = {i: [] for i in range(len(uniq_spike_clusters))}


for i, cluster in enumerate(uniq_spike_clusters):
    idx = np.where(spike_clusters == cluster)
    unit_kilo_frame  = spike_times[idx]

    unit_spike_time = APtimestamps[unit_kilo_frame]
    
    cluster_id[i] = cluster
    spiketimes[i].extend(unit_spike_time)
    
    depth[i] = cluster_group["depth"][i]
    group[i] = int(cluster_group.loc[cluster_group["cluster_id"] == cluster, "group"].values[0] == "good")

spiketimes = np.array([spiketimes[i] for i in range(len(uniq_spike_clusters))], dtype=object)
units = {
    'depth': depth,
    'cluster_id': cluster_id,
    'groups': group,
    'spiketimes': spiketimes
    'cluster_group': cluster_group
}

scipy.io.savemat(save_path, {'units': units})

print(f"Data saved at '{save_path}'.")

Data saved at 'D:\20250313\zarya_20250313__bankAB_unit.mat'.


In [28]:
## 20241220 version

depth = np.zeros(len(uniq_spike_clusters))
group = np.zeros(len(uniq_spike_clusters))
cluster_id = np.zeros(len(uniq_spike_clusters))
ch = np.zeros(len(uniq_spike_clusters))
spiketimes = {i: [] for i in range(len(uniq_spike_clusters))}

for i, cluster in enumerate(uniq_spike_clusters):
    idx = spike_clusters == cluster
    unit_kilo_frame  = spike_times[idx]
    # trial_frame = (unit_kilo_frame >= newtrial_frame)
    # unit_kilo_frame = unit_kilo_frame[trial_frame]

    unit_spike_time = APtimestamps[unit_kilo_frame]
    
    cluster_id[i] = cluster
    spiketimes[i].extend(unit_spike_time)
    
    chbest = chan_best[i]
    ch[i] = chbest
    depth[i] = channel_y_positions[chbest]
    group[i] = 

spiketimes = np.array([spiketimes[i] for i in range(len(uniq_spike_clusters))], dtype=object)
units = {
    'depth': depth,
    'ch': ch,
    'cluster_id': cluster_id,
    'spiketimes': spiketimes
}

scipy.io.savemat(save_path, {'units': units})

print(f"Data saved at '{save_path}'.")

ValueError: operands could not be broadcast together with shapes (2117,) (0,) 